<div class="practice--panel--ejs8J"><div data-purpose="practice-component-text" class="rt-scaffolding"><h4>Objective: Demonstrate virtual warehouse cache reuse</h4><p>This lab will walk you through a demonstration of how Snowflake uses a virtual warehouse cache to improve query performance.</p><p><br></p><p>1. Let us start by logging into the Snowflake Web UI. Ensure that any virtual warehouses that exist on your system are suspended. The purpose is to ensure that the virtual warehouse cache is cleared up.</p><p><br></p><p>2. Now, In the worksheet view, run the following complex SQL against sample data provided by Snowflake. This query will take around 5 minutes on an X-Small virtual warehouse.</p><div class="ud-component--base-components--code-block"><div><pre class="prettyprint linenums prettyprinted" role="presentation" style=""><ol class="linenums"><li class="L0"><span class="pln">USE DATABASE SNOWFLAKE_SAMPLE_DATA</span><span class="pun">;</span></li><li class="L1"><span class="pln">USE SCHEMA SNOWFLAKE_SAMPLE_DATA</span><span class="pun">.</span><span class="pln">TPCDS_SF10TCL</span><span class="pun">;</span></li><li class="L2"><span class="kwd">select</span><span class="pln"> dt</span><span class="pun">.</span><span class="pln">d_year</span></li><li class="L3"><span class="pun">,</span><span class="pln">sum</span><span class="pun">(</span><span class="pln">ss_ext_discount_amt</span><span class="pun">)</span><span class="pln"> sum_agg</span></li><li class="L4"><span class="kwd">from</span><span class="pln"> date_dim dt</span></li><li class="L5"><span class="pun">,</span><span class="pln">store_sales</span></li><li class="L6"><span class="kwd">where</span><span class="pln"> dt</span><span class="pun">.</span><span class="pln">d_date_sk </span><span class="pun">=</span><span class="pln"> store_sales</span><span class="pun">.</span><span class="pln">ss_sold_date_sk</span></li><li class="L7"><span class="kwd">and</span><span class="pln"> dt</span><span class="pun">.</span><span class="pln">d_moy</span><span class="pun">=</span><span class="lit">1</span></li><li class="L8"><span class="kwd">group</span><span class="pln"> </span><span class="kwd">by</span><span class="pln"> dt</span><span class="pun">.</span><span class="pln">d_year</span></li><li class="L9"><span class="pln">order </span><span class="kwd">by</span><span class="pln"> dt</span><span class="pun">.</span><span class="pln">d_year</span></li><li class="L0"><span class="pun">,</span><span class="pln">sum_agg asc</span><span class="pun">;</span></li></ol></pre></div></div><p><br></p><p>3. After the query is complete, note down the time taken by the query and review the query plan for the query, the query profile will show that zero percent of data was scanned from the cache, indicating that the virtual warehouse cache had no data in it initially. Note that your results may differ slightly from the query profile below.</p><figure><img width="245" src="https://github.com/melwinmpk/Snowflake/assets/25386607/c2bd20c1-fba9-460b-9a86-d9690dc88172" height="424"></figure><p><br></p><p>4. Make sure you perform this next step soon after the previous query is run because we don't want the virtual warehouse to auto-suspend and lose its cache.</p><p><br></p><p>5. Let's run a slightly different query. Note that we have changed one of the conditions to ensure that the query does not get answered by the query result cache, but we have kept it similar enough so that it can use data that exists in the virtual warehouse cache already.</p><div class="ud-component--base-components--code-block"><div><pre class="prettyprint linenums prettyprinted" role="presentation" style=""><ol class="linenums"><li class="L0"><span class="pln">USE DATABASE SNOWFLAKE_SAMPLE_DATA</span><span class="pun">;</span></li><li class="L1"><span class="pln">USE SCHEMA SNOWFLAKE_SAMPLE_DATA</span><span class="pun">.</span><span class="pln">TPCDS_SF10TCL</span><span class="pun">;</span></li><li class="L2"><span class="kwd">select</span><span class="pln"> dt</span><span class="pun">.</span><span class="pln">d_year</span></li><li class="L3"><span class="pun">,</span><span class="pln">sum</span><span class="pun">(</span><span class="pln">ss_ext_discount_amt</span><span class="pun">)</span><span class="pln"> sum_agg</span></li><li class="L4"><span class="kwd">from</span><span class="pln"> date_dim dt</span></li><li class="L5"><span class="pun">,</span><span class="pln">store_sales</span></li><li class="L6"><span class="kwd">where</span><span class="pln"> dt</span><span class="pun">.</span><span class="pln">d_date_sk </span><span class="pun">=</span><span class="pln"> store_sales</span><span class="pun">.</span><span class="pln">ss_sold_date_sk</span></li><li class="L7"><span class="kwd">and</span><span class="pln"> dt</span><span class="pun">.</span><span class="pln">d_moy</span><span class="pun">=</span><span class="lit">12</span></li><li class="L8"><span class="kwd">group</span><span class="pln"> </span><span class="kwd">by</span><span class="pln"> dt</span><span class="pun">.</span><span class="pln">d_year</span></li><li class="L9"><span class="pln">order </span><span class="kwd">by</span><span class="pln"> dt</span><span class="pun">.</span><span class="pln">d_year</span></li><li class="L0"><span class="pun">,</span><span class="pln">sum_agg asc</span><span class="pun">;</span></li></ol></pre></div></div><p><br></p><p>6. After the query is complete, note down the time taken by the query and review the query plan for the query, the query profile will show that almost 80% of data was scanned from the cache, indicating that the virtual warehouse cache was reused. Also, note that the time taken by this query is shorter than the previous query because a high percentage of data was already available in the virtual warehouse cache, thus reducing remote disk I/O. Note that your results may differ from the query profile below depending on the virtual warehouse configuration that you are using and other factors.</p><figure><img width="248" src="https://github.com/melwinmpk/Snowflake/assets/25386607/82c0a477-c152-44f4-a52d-baa6f1a8aa51" height="458"></figure><p><br></p><p>7. Let's suspend our virtual warehouse. You can do so through the web interface. Once the virtual warehouse is suspended, re-run a slightly different query. Note that we have changed one of the conditions to ensure that the query result cache does not answer the query. Also, since we have suspended the virtual warehouse, so the virtual warehouse cache has been purged and shouldn't be used by this following query.</p><div class="ud-component--base-components--code-block"><div><pre class="prettyprint linenums prettyprinted" role="presentation" style=""><ol class="linenums"><li class="L0"><span class="pln">USE DATABASE SNOWFLAKE_SAMPLE_DATA</span><span class="pun">;</span></li><li class="L1"><span class="pln">USE SCHEMA SNOWFLAKE_SAMPLE_DATA</span><span class="pun">.</span><span class="pln">TPCDS_SF10TCL</span><span class="pun">;</span></li><li class="L2"><span class="kwd">select</span><span class="pln"> dt</span><span class="pun">.</span><span class="pln">d_year</span></li><li class="L3"><span class="pun">,</span><span class="pln">sum</span><span class="pun">(</span><span class="pln">ss_ext_discount_amt</span><span class="pun">)</span><span class="pln"> sum_agg</span></li><li class="L4"><span class="kwd">from</span><span class="pln"> date_dim dt</span></li><li class="L5"><span class="pun">,</span><span class="pln">store_sales</span></li><li class="L6"><span class="kwd">where</span><span class="pln"> dt</span><span class="pun">.</span><span class="pln">d_date_sk </span><span class="pun">=</span><span class="pln"> store_sales</span><span class="pun">.</span><span class="pln">ss_sold_date_sk</span></li><li class="L7"><span class="kwd">and</span><span class="pln"> dt</span><span class="pun">.</span><span class="pln">d_moy</span><span class="pun">=</span><span class="lit">11</span></li><li class="L8"><span class="kwd">group</span><span class="pln"> </span><span class="kwd">by</span><span class="pln"> dt</span><span class="pun">.</span><span class="pln">d_year</span></li><li class="L9"><span class="pln">order </span><span class="kwd">by</span><span class="pln"> dt</span><span class="pun">.</span><span class="pln">d_year</span></li><li class="L0"><span class="pun">,</span><span class="pln">sum_agg asc</span><span class="pun">;</span></li></ol></pre></div></div><p><br></p><p>8. After the query is complete, note down the time taken by the query and review the query plan for the query, the query profile will show that NONE of the data was scanned from the cache. Also, note that the time taken by this query is a little longer than the previous one because it could not benefit from the virtual warehouse cache since it was purged after the virtual warehouse was suspended.</p><figure><img width="242" src="https://github.com/melwinmpk/Snowflake/assets/25386607/ec298f8d-6627-4dcf-b614-438d6b6e5334" height="432"></figure></div><div class="practice--question-list--1HR9g practice--mt-md--3j-aS" data-purpose="question-list"><h4 class="ud-heading-md practice--mb-xs--Lpg-b">Questions for this assignment</h4><div><div class="question--question--152Yi"><div class="ud-text-bold question--title--2FD-U"><div data-purpose="safely-set-inner-html:rich-text-viewer:html" class="question--title-text--2kBEO rt-scaffolding"><p>After a virtual warehouse is suspended and resumed, a new query can not take advantage of the warehouse cache. What is the reason behind this?</p></div></div><div class="question--answer--2ASXa"></div></div></div></div></div>

<div class="question--question--152Yi"><div class="ud-text-bold question--title--2FD-U"><div data-purpose="safely-set-inner-html:rich-text-viewer:html" class="question--title-text--2kBEO rt-scaffolding"><p>After a virtual warehouse is suspended and resumed, a new query can not take advantage of the warehouse cache. What is the reason behind this?</p></div></div><div class="question--answer--2ASXa"><div data-purpose="safely-set-inner-html:submission:user-answer-body"><p>When a virtual warehouse is suspended, all its compute and temporary storage resources related are released or de-provisioned. Thus, the cache stored on the temporary storage gets purged and is no longer available when the virtual warehouse is resumed. After a virtual warehouse is resumed, the cache gets rebuilt over time as queries execute.</p></div></div></div>